# Exercise 03 : Aggregations

In [198]:
import pandas as pd
import sqlite3

In [199]:
con = sqlite3.connect('../data/checking-logs.sqlite')

In [200]:
pd.read_sql("PRAGMA table_info(checker);", con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [201]:
pd.read_sql('SELECT * FROM test LIMIT 10', con)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [202]:
df_min = pd.read_sql("""
                     SELECT t.uid,
                            MIN((strftime('%s', DATETIME(t.first_commit_ts)) - d.deadlines) / 3600) AS delta_hours
                     FROM test t
                              JOIN deadlines d ON t.labname = d.labs
                     WHERE t.labname != 'project1'
                     GROUP BY t.uid, t.labname
                     ORDER BY delta_hours
                         LIMIT 1
                     """, con)

In [203]:
df_min

,uid,delta_hours
0,user_30,-202


In [204]:
df_max = pd.read_sql("""
                     SELECT t.uid,
                            MIN((strftime('%s', DATETIME(t.first_commit_ts)) - d.deadlines) / 3600) AS delta_hours
                     FROM test t
                              JOIN deadlines d ON t.labname = d.labs
                     WHERE t.labname != 'project1'
                     GROUP BY t.uid, t.labname
                     ORDER BY delta_hours DESC
                         LIMIT 1
                     """, con)

In [205]:
df_max

,uid,delta_hours
0,user_25,-2


In [206]:
df_avg = pd.read_sql("""
                     SELECT AVG((strftime('%s', DATETIME(t.first_commit_ts)) - d.deadlines) / 3600) AS delta_hours
                     FROM test t
                              JOIN deadlines d ON t.labname = d.labs
                     WHERE t.labname != 'project1'
                     """, con)

In [207]:
df_avg

,delta_hours
0,-89.125


In [208]:
views_diff = pd.read_sql("""
                         SELECT t.uid,
                                AVG((strftime('%s', t.first_commit_ts) - d.deadlines) / 3600.0) AS avg_diff,
                                (SELECT COUNT(*) FROM pageviews p WHERE p.uid = t.uid)          AS pageviews
                         FROM test t
                                  JOIN deadlines d ON t.labname = d.labs
                         WHERE t.labname != 'project1'
                         GROUP BY t.uid
                         """, con)

In [209]:
views_diff

,uid,avg_diff,pageviews
0,user_1,-65.119778,28
1,user_10,-75.242444,89
2,user_14,-159.568796,143
3,user_17,-62.207667,47
4,user_18,-6.368148,3
5,user_19,-99.440417,16
6,user_21,-96.111181,10
7,user_25,-93.474944,179
8,user_28,-86.793833,149
9,user_3,-105.738222,317


In [210]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.279143
pageviews,-0.279143,1.000000


In [211]:
con.close()